# Project Notebook
Generated from script.

In [1]:
import sys
import os
from pathlib import Path

# Add project root to sys.path
# Assuming notebook is in 'notebooks/' and project root is one level up
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
from pokemon_predictor import config
from pokemon_predictor.data_utils import load_data
import pandas as pd
import numpy as np
import joblib


# Quantitative Model Evaluation

In [2]:
from pokemon_predictor.data_utils import load_data
from pokemon_predictor.visualization import plot_model_comparison, plot_feature_importance, plot_class_distribution_comparison
import joblib
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, precision_score, recall_score
from pokemon_predictor import config
from pokemon_predictor.model_utils import FocalLoss

# 1. Load Test Data
X_test_xgb, _, y_test, _, classes = load_data('rgb', split_data=True)
_, X_test_mlp, _, _, _ = load_data('hist', split_data=True)
# Hybrid data alignment needs care in a real scenario, here we rely on split randomness being seeded identical

print("Models Loading...")
models = {}
try:
    models['XGBoost'] = joblib.load(config.MODELS_DIR / "xgboost_model_advanced.pkl")
except:
    pass
    
try:
    models['MLP_Opt'] = load_model(config.MODELS_DIR / "mlp_model_optimized.h5")
    models['MLP_Opt_Thresh'] = joblib.load(config.MODELS_DIR / "best_threshold.pkl")
except:
    pass

# 2. Evaluate
metrics = []
preds = {}

if 'XGBoost' in models:
    y_pred = models['XGBoost'].predict(X_test_xgb)
    preds['XGBoost'] = y_pred
    f1 = f1_score(y_test, y_pred, average='macro')
    metrics.append({'Model': 'XGBoost', 'F1': f1})
    
    # Feature Importance
    # Taking average of estimators for MultiOutput
    # (Simplified for notebook)
    pass 

if 'MLP_Opt' in models:
    probs = models['MLP_Opt'].predict(X_test_mlp, verbose=0)
    thresh = models.get('MLP_Opt_Thresh', 0.5)
    y_pred = (probs > thresh).astype(int)
    preds['MLP_Opt'] = y_pred
    f1 = f1_score(y_test, y_pred, average='macro')
    metrics.append({'Model': 'MLP (Opt)', 'F1': f1})

# 3. Plot Comparison
if metrics:
    df_metrics = pd.DataFrame(metrics)
    plot_model_comparison(df_metrics)


Models Loading...
